In [23]:
using StatsBase, Distributions, Plotly, DataFrames, Plots, PyPlot
plotly()
include("episiming.jl");

In [2]:
function powerDecay(a::Number, b::Number, x)
    y = rowWiseNorm(x)
    return 1.0 ./ (1.0 .+ (y ./ a) .^ b)
end

powerDecay (generic function with 1 method)

In [3]:
censoResidencial = [.21, .26, .20, .17, .08, .04, .02, 0.02];

In [4]:
geradorCenso(x) = geraResidencias(x, censoResidencial)
geradorDensidade(k) = (x) -> geraResidencias(x, k)

geradorDensidade (generic function with 1 method)

# Pop350

In [5]:
distribuicao = [
    16 11  0  0  0 6  4  8  8 6;
    10 12 12  6  8 9  8  6  7 5;
     0 10 14 10 12 8  0  0  6 8;
     0 12 10 14 11 9  0  0  5 7;
     9 11  0 12 10 7  8  7  8 0;
];

nPessoas = sum(distribuicao);

I0 = 8

fKernel(x) = powerDecay(1., 1.5, x)

fKernel (generic function with 1 method)

In [14]:
pop350 = distribuiPessoas(distribuicao, geradorCenso, I0, 5, fKernel);

In [15]:
data = Dict([
    "x" => pop350.pessoas.posicao[:, 1],
    "autobinx" => false,
    "xbins" => Dict([
        "start" => 0,
        "end" => size(distribuicao)[2],
        "size" => 1
    ]),
    "y"=>pop350.pessoas.posicao[:, 2],
    "autobiny" => false,
    "ybins" => Dict([
        "start" => 0,
                "end" => size(distribuicao)[1],
        "size" => 1
    ]),
]);

Plotly.plot(Plotly.histogram2d(data))

data: [
  "histogram2d with fields autobinx, autobiny, type, x, xbins, y, and ybins"
]

layout: "layout with field margin"

In [16]:
data = Dict([
    "x"=>pop350.pessoas.posicao[:, 1],
    "y"=>pop350.pessoas.posicao[:, 2]
]);

Plotly.plot(Plotly.histogram2dcontour(data))

data: [
  "histogram2dcontour with fields type, x, and y"
]

layout: "layout with field margin"

In [17]:
# ρForma = 0.2 # shape factor of gamma distribution
# ρEscala = 5 # scale (mean value = scale * shape)
# ρPop = rand(Gamma(ρForma, ρEscala), nPessoas)
αᵣ = 0.8
βᵣ = 0.26
βₛ = 0.10
βᵤ = 0.06
γ = 0.1;

In [24]:
txTransmissaoR = βᵣ ./ [pop350.residencias[i].n ^ αᵣ for i in pop350.pessoas.residencias];
txTransmissaoS = βₛ ./ [pop350.sociais[i].n for i in pop350.pessoas.social];
txTransmissaoG = βᵤ ./ calculaDistancia(pop350, ones(Bool, pop350.n), ones(Bool, pop350.n), fKernel);

In [ ]:
@time (S, I, R) = evolucaoMista(pop350, Array(1:30), 50, txTransmissaoR, txTransmissaoS, txTransmissaoG, γ, fKernel);

In [ ]:
Plots.plot(mean(S, dims=1)[1, :], label = "Suscetíveis")
Plots.plot!(mean(I, dims=1)[1, :], label = "Infectados")
Plots.plot!(mean(R, dims=1)[1, :], label = "Recuperados")